In [ ]:
from torch.utils.cpp_extension import load
import torch

In [ ]:
bnets = load(name='bayesian_nets',
             sources=['bayesian_nets.cc'],
             extra_include_paths=['../../include'],
             extra_cflags=['-Wall -Wextra -Wpedantic -O3 -std=c++1z'],
             verbose=True)

In [ ]:
help(bnets.run_ghmc)

In [ ]:
sample = bnets.run_ghmc()

In [ ]:
torch.std_mean(sample, 0, keepdim=True)